In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import base64
import io

# Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Exploratory 5D Data Visualization"), className="mb-2")
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("Upload Excel File"),
            dcc.Upload(
                id='upload-data',
                children=html.Div(['Drag and Drop or ', html.A('Select Files')]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                },
                multiple=False
            ),
            html.Label("Sheet Name"),
            dbc.Input(id='sheet-name', type='text', placeholder='Enter sheet name'),
            dbc.Button("Load Data", id='load-data-button', color='primary'),
            html.Div(id='output-data-upload')
        ])
    ]),
    dbc.Row([
        dbc.Col([
            html.H2("Statistical Overview"),
            html.Div(id='data-summary', style={'margin-bottom': '20px'})
        ])
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("X-axis"),
            dcc.Dropdown(id='x-axis-dropdown'),
            html.Label("Y-axis"),
            dcc.Dropdown(id='y-axis-dropdown'),
            html.Label("Z-axis"),
            dcc.Dropdown(id='z-axis-dropdown'),
            html.Label("Color"),
            dcc.Dropdown(id='color-dropdown'),
            html.Label("Size"),
            dcc.Dropdown(id='size-dropdown'),
            html.Div(id='size-legend'),
            html.Label("Input Outlier Line Numbers (comma separated):", style={'margin-top': '20px'}),
            dbc.Input(id='outlier-lines', type='text', placeholder='e.g., 1, 3, 7'),
            dbc.Button("Remove Outliers", id='remove-outliers-button', color='danger', style={'margin-top': '10px'})
        ], width=3),
        dbc.Col(dcc.Graph(id='3d-scatter-plot', style={'height': '80vh', 'width': '100%'}), width=9)
    ]),
    dbc.Row([
        dbc.Col([
            html.H2("Prediction Plane Intervals"),
            
            html.Label("X-axis range:"),
            dbc.Input(id='x-range-min', type='number', placeholder='Min', debounce=True),
            dbc.Input(id='x-range-max', type='number', placeholder='Max', debounce=True),
            
            html.Label("Z-axis range:"),
            dbc.Input(id='z-range-min', type='number', placeholder='Min', debounce=True),
            dbc.Input(id='z-range-max', type='number', placeholder='Max', debounce=True),
            
            html.Label("Color-axis range:"),
            dbc.Input(id='color-range-min', type='number', placeholder='Min', debounce=True),
            dbc.Input(id='color-range-max', type='number', placeholder='Max', debounce=True),
            
            html.Label("Size-axis range:"),
            dbc.Input(id='size-range-min', type='number', placeholder='Min', debounce=True),
            dbc.Input(id='size-range-max', type='number', placeholder='Max', debounce=True),
            
            dbc.Button("Draw Plane", id='draw-plane-button', color='primary'),
        ])
    ])
], fluid=True)


df = pd.DataFrame()

@app.callback(
    Output('output-data-upload', 'children'),
    Output('x-axis-dropdown', 'options'),
    Output('y-axis-dropdown', 'options'),
    Output('z-axis-dropdown', 'options'),
    Output('color-dropdown', 'options'),
    Output('size-dropdown', 'options'),
    Output('data-summary', 'children'),
    Input('load-data-button', 'n_clicks'),
    State('upload-data', 'contents'),
    State('sheet-name', 'value')
)
def load_data(n_clicks, contents, sheet_name):
    if n_clicks is None or contents is None:
        return '', [], [], [], [], [], ''
    
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    global df
    df = pd.read_excel(io.BytesIO(decoded), sheet_name=sheet_name)
    
    options = [{'label': col, 'value': col} for col in df.columns]
    summary = df.describe().reset_index().to_dict('records')
    
    table_header = [html.Thead(html.Tr([html.Th(col) for col in df.describe().reset_index().columns]))]
    table_body = [html.Tbody([html.Tr([html.Td(value) for value in row.values()]) for row in summary])]
    
    summary_table = dbc.Table(table_header + table_body, bordered=True, striped=True, hover=True)
    
    return 'Data Loaded', options, options, options, options, options, summary_table

@app.callback(
    Output('3d-scatter-plot', 'figure'),
    [
        Input('x-axis-dropdown', 'value'),
        Input('y-axis-dropdown', 'value'),
        Input('z-axis-dropdown', 'value'),
        Input('color-dropdown', 'value'),
        Input('size-dropdown', 'value'),
        Input('remove-outliers-button', 'n_clicks'),
        Input('draw-plane-button', 'n_clicks')
    ],
    [
        State('outlier-lines', 'value'),
        State('x-range-min', 'value'),
        State('x-range-max', 'value'),
        State('z-range-min', 'value'),
        State('z-range-max', 'value'),
        State('color-range-min', 'value'),
        State('color-range-max', 'value'),
        State('size-range-min', 'value'),
        State('size-range-max', 'value')
    ]
)
def update_graph(x_axis, y_axis, z_axis, color, size, remove_n_clicks, update_n_clicks, outlier_lines, x_min, x_max, z_min, z_max, color_min, color_max, size_min, size_max):
    if df.empty or not all([x_axis, y_axis, z_axis, color, size]):
        return go.Figure()

    df_filtered = df.copy()


    if outlier_lines and remove_n_clicks:
        try:
            #df = df.iloc[1:]
            outlier_indices = [int(i.strip()) - 1 for i in outlier_lines.split(',')]  

            df_filtered = df_filtered.drop(df_filtered.index[outlier_indices])
        except ValueError:
            pass  

    fig = px.scatter_3d(
        df_filtered,
        x=x_axis,
        y=y_axis,
        z=z_axis,
        color=color,
        size=size,
        title="3D Scatter Plot"
    )
    fig.update_layout(scene=dict(
        xaxis_title=x_axis,
        yaxis_title=str(y_axis), #'Viscosity at ' + str(y_axis),
        zaxis_title=z_axis
    ))

    if all(v is not None for v in [x_min, x_max, z_min, z_max, color_min, color_max, size_min, size_max]):
        X = df_filtered[[x_axis, z_axis, color, size]]
        y = df_filtered[y_axis]
        model = RandomForestRegressor()
        model.fit(X, y)

        x_range = np.linspace(x_min, x_max, 10)
        z_range = np.linspace(z_min, z_max, 10)
        color_range = np.linspace(color_min, color_max, 10)
        size_range = np.linspace(size_min, size_max, 10)
        xx, zz, cc, ss = np.meshgrid(x_range, z_range, color_range, size_range)
        X_pred = np.c_[xx.ravel(), zz.ravel(), cc.ravel(), ss.ravel()]
        y_pred = model.predict(X_pred)
        yy = y_pred.reshape(xx.shape)

        #for trace in fig.data:
        #    if isinstance(trace, go.Surface):
        #        fig.data = tuple(t for t in fig.data if t != trace)

        # Add plane to plot
        fig.add_trace(
            go.Surface(
                x=xx[:, :, 0, 0],
                y=yy[:, :, 0, 0],
                z=zz[:, :, 0, 0],
                opacity=0.5,
                colorscale=[[0, 'red'], [1, 'red']],
                showscale=False
            )
        )
        
    return fig

@app.callback(
    Output('size-legend', 'children'),
    [Input('size-dropdown', 'value')]
)
def update_size_legend(size):
    if df.empty or size is None:
        return ""
    
    min_size = df[size].min()
    max_size = df[size].max()
    return html.Div([
        html.Hr(), 
        html.Label(f"Minimum size variable value: {min_size}"),
        html.Br(),
        html.Label(f"Maximum size variable value: {max_size}")
    ])

if __name__ == '__main__':
    app.run_server(debug=True)

#http://127.0.0.1:8050/


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~\anaconda3\lib\site-packages\plotly\express\_chart_types.py in scatter_3d(
    data_frame=    MgO=  Al2O3=       CaO=      SiO2=  Viscosit...     1.08               2.23                NaN  ,
    x='MgO=',
    y='SiO2',
    z='CaO',
    color='Al2O3',
    symbol=None,
    size='Viscosity at 1500',
    text=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    error_z=None,
    error_z_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    size_max=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    color_continuous_scale=None,
    range_color=None,
    color_continuous_midpoint=None,
    symbol_sequence=None,
    symbol_map=None,
    opacity=None,
    log_x=

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~\anaconda3\lib\site-packages\plotly\express\_chart_types.py in scatter_3d(
    data_frame=    MgO=  Al2O3=       CaO=      SiO2=  Viscosit...     1.08               2.23                NaN  ,
    x='MgO=',
    y='SiO2',
    z='Al2O3=',
    color='Al2O3',
    symbol=None,
    size='Viscosity at 1500',
    text=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    error_z=None,
    error_z_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    size_max=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    color_continuous_scale=None,
    range_color=None,
    color_continuous_midpoint=None,
    symbol_sequence=None,
    symbol_map=None,
    opacity=None,
    log

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~\anaconda3\lib\site-packages\plotly\express\_chart_types.py in scatter_3d(
    data_frame=    MgO=  Al2O3=       CaO=      SiO2=  Viscosit...     1.08               2.23                NaN  ,
    x='MgO=',
    y='SiO2',
    z='Al2O3=',
    color='Al2O3',
    symbol=None,
    size='SiO2=',
    text=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    error_z=None,
    error_z_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    size_max=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    color_continuous_scale=None,
    range_color=None,
    color_continuous_midpoint=None,
    symbol_sequence=None,
    symbol_map=None,
    opacity=None,
    log_x=False,
  

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~\anaconda3\lib\site-packages\plotly\express\_chart_types.py in scatter_3d(
    data_frame=    MgO=  Al2O3=       CaO=      SiO2=  Viscosit...     1.08               2.23                NaN  ,
    x='MgO=',
    y='SiO2',
    z='Al2O3=',
    color='Viscosity at 1500',
    symbol=None,
    size='SiO2=',
    text=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    error_z=None,
    error_z_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    size_max=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    color_continuous_scale=None,
    range_color=None,
    color_continuous_midpoint=None,
    symbol_sequence=None,
    symbol_map=None,
    opacity=None,
    log